In [2]:
from transformers import BertTokenizer, BertModel

In [83]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
model = BertModel.from_pretrained("bert-large-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

KeyboardInterrupt: 

In [79]:
text = 'if there is a (non-deterministic)\ninput asked from the user, and a random vertex if the evolution depends on a stochastic\nenvironment. An optimal strategy for Min can then be used as the basis for the synthesis\nof a “good” driver, i.e. one which minimises the probability of entering the error state\nindependently of the behaviour of the user.\nThe main algorithmic problem about SSGs is the computation of values of the vertices\nand optimal strategies for the players. This problem was first adressed by Condon, who\nshowed that deciding whether the value of a vertex is greater than 1\n2 belongs to NP and\nco-NP [Con92].\nCondon’s algorithm guesses non-deterministically the values of vertices,\nwhich are rational numbers of linear size, and checks that they are solutions of some local\noptimality equations. This algorithm is correct only for stopping games, where the pebble\nreaches either the target or a sink target with probability one, regardless of the players’\nstrategies.\nAny SSG can be transformed in polynomial time into a stopping SSG with\n(almost) the same values, but it incurs a quadratic blow-up of the size of the game.\nThree other algorithms for solving SSGs are presented in [Con93]. The first one com-\nputes the values of the vertices using a quadratic program with linear constraints. The\nsecond one computes iteratively from below the values of the vertices, and the third is a\nstrategy improvement algorithm `\na la Hoffman-Karp [HK66]. The two latter algorithms,\nas the ones recently proposed in [Som05], solve a series of linear programs which could be\nof exponential length. Furthermore, solving a linear program requires high-precision arith-\nmetic, even if it can be done in polynomial time [Kha79, Ren88]. The best randomised\nalgorithms achieve sub-exponential expected time eO(√n) [Lud95, Hal07].\nIn this paper we present two algorithms computing the values and optimal strategies\nin SSGs: the “permutation-enumeration” and the “permutation-improvement” algorithms.\nThe common basis for both algorithms is that optimal strategies can be looked for in a\nsubset of the positional strategies called permutation strategies. Permutation strategies are\nderived from permutations over the random vertices. In order to find optimal strategies, the\npermutation-enumeration algorithm performs an exhaustive search among all permutation\nstrategies, whereas the permutation-improvement algorithm performs successive improve-\nments of permutation strategies, `\na la Hoffman-Karp [HK66].'
len(text.split())
te = "I went to the store."

In [82]:
encoded_input = tokenizer(te, return_tensors='pt')
print(encoded_input.input_ids)


tensor([[ 101, 1045, 2253, 2000, 1996, 3573, 1012,  102]])


In [13]:
output = model(**encoded_input)
#print(output)
print(output.pooler_output.shape)

torch.Size([1, 768])


In [21]:
import json
import torch

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [14]:
def tokenize_entry(line):
    entry = json.loads(line)
    abstract = entry["abstract"]
    title = entry["title"]
    joined = title + ": " + abstract
    tokenized = tokenizer(joined, return_tensors="pt")

In [41]:
def generate_chunks(encoded):
    chunk_size = 512  # BERT's maximum input size

    input_ids = encoded_input['input_ids'].squeeze(0)  # Remove the batch dimension
    attention_mask = encoded_input['attention_mask'].squeeze(0)

    input_ids_chunks = []
    attention_mask_chunks = []

    for i in range(0, len(input_ids), chunk_size - 2):  # Account for special tokens
        # Extract segments of the tokenized input
        input_ids_chunk = input_ids[i:i + chunk_size - 2]
        attention_mask_chunk = attention_mask[i:i + chunk_size - 2]

        # Add special tokens ([CLS] and [SEP]) to the chunks
        input_ids_chunk = torch.cat([
            torch.tensor([tokenizer.cls_token_id]),  # [CLS] at the beginning
            input_ids_chunk,
            torch.tensor([tokenizer.sep_token_id])  # [SEP] at the end
        ])
        attention_mask_chunk = torch.cat([
            torch.tensor([1]),  # Attention mask for [CLS]
            attention_mask_chunk,
            torch.tensor([1])  # Attention mask for [SEP]
        ])

        # Pad the chunks to the chunk size
        padding_length = chunk_size - input_ids_chunk.size(0)
        if padding_length > 0:
            input_ids_chunk = torch.cat([input_ids_chunk, torch.zeros(padding_length, dtype=torch.long)])
            attention_mask_chunk = torch.cat([attention_mask_chunk, torch.zeros(padding_length, dtype=torch.long)])

        input_ids_chunks.append(input_ids_chunk.unsqueeze(0))  # Add batch dimension
        attention_mask_chunks.append(attention_mask_chunk.unsqueeze(0))  # Add batch dimension

    input_ids_chunks = torch.cat(input_ids_chunks, dim=0)
    attention_mask_chunks = torch.cat(attention_mask_chunks, dim=0)

    return input_ids_chunks, attention_mask_chunks

In [42]:
inp, att = generate_chunks(encoded_input)

In [66]:
inp.shape, att.shape
for n in range(inp.shape[0]):
    print(inp[n].shape)

torch.Size([512])
torch.Size([512])


In [100]:
def generate_embedding(input_chunks, attention_chunks):
    embeddings = []
    with torch.no_grad():
        for input_chunk, attention_chunk in zip(input_chunks, attention_chunks): # tensor [512] & tensor [512]
            embeddings.append(model(input_ids=input_chunk.unsqueeze(0), attention_mask=attention_chunk.unsqueeze(0))# Model expects [1,512] shape
                            .pooler_output[0]) # pooler_output.shape = [1, 768]

    if len(embeddings) == 1: 
        return embeddings[0]

    return torch.mean(torch.stack(embeddings), dim=0)       
    

In [102]:
embedding = generate_embedding(inp, att)
embedding.numpy()

array([-0.6220543 , -0.33725995, -0.9074133 ,  0.28864622,  0.6986108 ,
       -0.16639835,  0.0815863 ,  0.13572222, -0.7121061 , -0.9999281 ,
       -0.336446  ,  0.64945245,  0.9325261 ,  0.3047979 ,  0.37368822,
       -0.39301878,  0.37734118, -0.35326958,  0.18880358,  0.5457956 ,
        0.25987962,  0.9999927 , -0.01962734,  0.24808371,  0.20495851,
        0.7610008 , -0.45595765,  0.6455927 ,  0.86710995,  0.66149855,
       -0.11816373,  0.02685865, -0.96441746, -0.14318499, -0.95499897,
       -0.97415763,  0.3213859 , -0.40492025, -0.03221555,  0.00517625,
       -0.67411816,  0.32913372,  0.9999748 , -0.3365166 ,  0.5826086 ,
       -0.0694285 , -0.9999058 ,  0.2498289 , -0.6484609 ,  0.8074957 ,
        0.780831  ,  0.8868296 ,  0.0550229 ,  0.35060847,  0.47958905,
       -0.5278735 , -0.1786366 ,  0.17421943, -0.3022453 , -0.4684335 ,
       -0.4151519 ,  0.21276051, -0.68606496, -0.7311013 ,  0.7853577 ,
        0.71325636, -0.0033311 , -0.3059155 , -0.09873156, -0.04

In [ ]:
def index_embedding(embedding, title, id, categories):
    # Add the embedding to the indexing system
    index.add(embedding.numpy())

In [ ]:
with open("./ml-papers.txt", "r") as f:
    for line in f:
        tokenized = tokenize_entry(line)
        inputs, attentions = generate_chunks(tokenized)
        embedding = generate_embedding(inputs, attentions)
        

In [104]:
list(range(0, 190, 50))

[0, 50, 100, 150]

In [105]:
[1,2,3][:7]

[1, 2, 3]